In [1]:
import os
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(1)


inputs = keras.Input(shape=(128, 128, 3), name="img")
x = layers.Conv2D(128, 3, activation="relu")(inputs)
x = layers.Conv2D(256, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(256, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(256, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(256, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation='softmax')(x)

model = keras.Model(inputs, outputs, name="toy_resnet")

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])


In [2]:
model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 126, 126, 12  3584        ['img[0][0]']                    
                                8)                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 124, 124, 25  295168      ['conv2d[0][0]']                 
                                6)                                                       

In [3]:
import numpy as np
x_train = np.load('x_train.npy')[:-1]
y_train = np.load('y_train.npy')[:-1]
x_test = np.load('x_valid.npy')
y_test = np.load('y_valid.npy')

x_train = x_train / 255
x_test = x_test / 255

y_train = keras.utils.to_categorical(y_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)

In [4]:
model.fit(x_train, y_train, batch_size=36, epochs=15, validation_split=0.2)

Epoch 1/15
14/14 [==============================] - 426s 31s/step - loss: 1.3193 - accuracy: 0.3885 - val_loss: 2.2653 - val_accuracy: 0.0000e+00
Epoch 2/15
14/14 [==============================] - 423s 30s/step - loss: 1.2090 - accuracy: 0.3517 - val_loss: 2.5016 - val_accuracy: 0.0000e+00
Epoch 3/15
14/14 [==============================] - 423s 30s/step - loss: 1.1277 - accuracy: 0.4417 - val_loss: 1.9624 - val_accuracy: 0.0000e+00
Epoch 4/15
14/14 [==============================] - 423s 30s/step - loss: 1.0559 - accuracy: 0.5992 - val_loss: 2.3283 - val_accuracy: 0.0000e+00
Epoch 5/15
14/14 [==============================] - 423s 30s/step - loss: 0.9284 - accuracy: 0.6176 - val_loss: 3.2187 - val_accuracy: 0.0000e+00
Epoch 6/15
14/14 [==============================] - 427s 31s/step - loss: 0.8080 - accuracy: 0.6360 - val_loss: 2.5404 - val_accuracy: 0.0000e+00
Epoch 7/15
14/14 [==============================] - 424s 30s/step - loss: 0.8145 - accuracy: 0.6155 - val_loss: 2.4433 - val

In [5]:
print(model.evaluate(x_test, y_test))

3/3 [==============================] - 25s 7s/step - loss: 1.1013 - accuracy: 0.4583
[1.1012564897537231, 0.4583333432674408]


In [6]:
model.save('lungs_model')

INFO:tensorflow:Assets written to: lungs_model\assets


INFO:tensorflow:Assets written to: lungs_model\assets
